In [1]:
import trimesh as tm
import os
import pyvista as pv
import numpy as np
import topogenesis as tg

In [2]:

lattice_path = os.path.relpath("../data/voxelized_envelope.csv")
occ_lattice = tg.lattice_from_csv(lattice_path) # the current occupation lattice

path = os.path.relpath('../data/my_envelope.obj') # building outer boundaries file path
buildingplot = tm.load(path) # load specified building boundary mesh (footprint extruded to max height, USER INPUT)

In [11]:
simple_array = np.arange(9).reshape(3,3)
np.roll(simple_array, (0,-1), (0,1))


array([[1, 2, 0],
       [4, 5, 3],
       [7, 8, 6]])

In [6]:
occ_shifted_down = np.roll(occ_lattice, (0,0,-1), (0,1,2))
roof_mask = (occ_lattice == True)*(occ_shifted_down==False)
roof_mask_lattice = tg.to_lattice(roof_mask, occ_lattice)

In [7]:
# create a step one moore neighbourhood
s_vn = tg.create_stencil("von_neumann", 1)

# set the center to 0
s_vn.set_index([0, 0, 0], 0)

# assign the sum function
s_vn.function = tg.sfunc.sum  # np.sum

s_vn

stencil([[[0, 0, 0],
          [0, 1, 0],
          [0, 0, 0]],

         [[0, 1, 0],
          [1, 0, 1],
          [0, 1, 0]],

         [[0, 0, 0],
          [0, 1, 0],
          [0, 0, 0]]])

In [8]:
occ_neigh_sum = occ_lattice.apply_stencil(s_vn)
corner_mask = occ_neigh_sum * occ_lattice == 3
corner_mask_lattice  = tg.to_lattice(corner_mask, occ_lattice)

In [10]:
# convert to trimesh definition, taken from spatial computing workshop
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter

# initiating the plotter
p = pv.Plotter(notebook=True)

corner_mask_lattice.fast_vis(p)
# adding the base mesh: light blue
#p.add_mesh(tri_to_pv(context), color='#abd8ff')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(285.89291929890817, 200.89291929890814, 270.89291929890817),
 (35.0, -50.0, 20.0),
 (0.0, 0.0, 1.0)]